In [ ]:
import sys, os
path2add = os.path.normpath(os.path.abspath(os.path.join('', os.path.pardir)))
if (not (path2add in sys.path)) :
    sys.path.append(path2add)

In [ ]:
#!/usr/bin/env python

# Copyright 2018 Paul Archer
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import time
import json
from datetime import datetime, timedelta, timezone
from sys import stdout
import argparse

from pvstats.pvinverter.factory import PVInverterFactory
from pvstats.report import PVReportFactory

import logging
import traceback

from astral import LocationInfo
from astral.sun import sun, daylight
from astral.geocoder import database, lookup

# Setup the logging
logging.basicConfig(format="%(asctime)s: %(levelname)s %(message)s")
_log = logging.getLogger()


# Load the configuration
def load_config(location):
  with open(location) as f:
    cfg = json.load(f)

  try:
    if cfg.get('log_level') is not None:
      _log.setLevel(cfg['log_level'])
  except:
    _log.setLevel('INFO')
  return cfg

# Determine the panel sleep time
def inverter_sleep(location_name, offset):

  location = lookup(location_name, database())
  now = datetime.now(location.tzinfo)
  s = sun(location.observer, date=now, tzinfo=location.timezone)    
    
  if now < s['noon']:
    _log.info('before noon')
    midday_offset = 0
  else:
    _log.info('after noon')
    midday_offset = 1    
  
  d = daylight(location.observer, date=now, tzinfo=location.timezone)  
  surise = d[0]
  sunrise_offset = surise - timedelta(minutes=offset)
  sunset = d[1]
  sunset_offset = sunset + timedelta(minutes=offset)

  dT = daylight(location.observer, date=now+timedelta(days=midday_offset), tzinfo=location.timezone)
  suriseT = dT[0]
  sunriseT_offset = suriseT - timedelta(minutes=offset)

  if not sunrise_offset <= now <= sunset_offset:
    
    _log.info(f'Sleeping till {sunriseT_offset.strftime("%c %z")}')  
    time.sleep((sunriseT_offset-now).seconds)
    return True
  
  else:
    _log.debug(f'Awake till {sunset_offset.strftime("%c %z")}')  
    return False


def main():
  # Parse input arguments
  parser = argparse.ArgumentParser(
      description="Photovoltaic Inverter Statistics Scanner and Uploader",
      prog="pvstats",
      usage="%(prog)s [options]")
  parser.add_argument("--cfg", help="Configuration File", nargs=1, default=["../pvstats.conf"])

  args, unknown = parser.parse_known_args()
  if unknown:
    _log.warning(f'Unknown arguments passed; {unknown}')
  # Initialise
  cfg = load_config(vars(args)['cfg'][0])

  # Get an PV inverter client
  inverter = PVInverterFactory(cfg['inverter']['model'], cfg['inverter'])

  # Create the report channels
  reports = []
  for rpt in cfg['reports']:
    _log.debug(json.dumps(rpt, sort_keys=True,
                         indent=4, separators=(',', ': '),default=str))
    r = PVReportFactory(rpt)
    if r != None:
      reports.append(r)

  sleep_offset = cfg.get('sleep_offset')
  location = cfg.get('location')
  while True:
    if sleep_offset and location:
      inverter_sleep(location, sleep_offset)
    tstart = datetime.now()
    try:
      # Grab the data from the inverter
      inverter.connect()
      inverter.read()

      # Log it
      _log.debug(json.dumps(inverter.registers, sort_keys=True,
                           indent=4, separators=(',', ': '),default=str))

      # Publish it
      for rpt in reports:
        rpt.publish(inverter.registers)

    except Exception as err:
      _log.debug(traceback.format_exc())
      _log.debug("Ignoring = {}".format(err))

    finally:
      inverter.close()

    tend = datetime.now()
    time.sleep(max((tstart - tend).total_seconds() + cfg['sample_period'],0))


if __name__ == "__main__":
  main()

# vim: set expandtab ts=2 sw=2:
